In [1]:
from datetime import datetime
import os
import re
import requests
import time
import urllib

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
top_actors = ['Dwayne Johnson',
'Chris Hemsworth',
'Robert Downey Jr.',
'Akshay Kumar',
'Jackie Chan',
'Bradley Cooper',
'Adam Sandler',
'Scarlett Johansson',
'Sofia Vergara',
'Chris Evans']

In [3]:
from imdb_helper_functions import *
from imdb_code import get_actors_by_movie_soup, get_movies_by_actor_soup, get_movie_distance

actors_url_dict = get_actor_url_by_name(top_actors)
actors_url_dict

{'Dwayne Johnson': 'https://www.imdb.com/name/nm0425005/',
 'Chris Hemsworth': 'https://www.imdb.com/name/nm1165110/',
 'Robert Downey Jr.': 'https://www.imdb.com/name/nm0000375/',
 'Akshay Kumar': 'https://www.imdb.com/name/nm0474774/',
 'Jackie Chan': 'https://www.imdb.com/name/nm0000329/',
 'Bradley Cooper': 'https://www.imdb.com/name/nm0177896/',
 'Adam Sandler': 'https://www.imdb.com/name/nm0001191/',
 'Scarlett Johansson': 'https://www.imdb.com/name/nm0424060/',
 'Sofia Vergara': 'https://www.imdb.com/name/nm0005527/',
 'Chris Evans': 'https://www.imdb.com/name/nm0262635/'}

In [4]:
save_actors, saved_movies = check_saved_data()

In [5]:
# print(get_movies_by_actor('https://www.imdb.com/name/nm0474774/'))
# print(get_actors_by_movie('https://www.imdb.com/title/tt4799050/'))

In [6]:
# actors_text[0][1], actors_text[1][1]
"""
{'Dwayne Johnson': 'https://www.imdb.com/name/nm0425005/',
 'Chris Hemsworth': 'https://www.imdb.com/name/nm1165110/',
 'Robert Downey Jr.': 'https://www.imdb.com/name/nm0000375/',
 'Akshay Kumar': 'https://www.imdb.com/name/nm0474774/',
 'Jackie Chan': 'https://www.imdb.com/name/nm0000329/',
 'Bradley Cooper': 'https://www.imdb.com/name/nm0177896/',
 'Adam Sandler': 'https://www.imdb.com/name/nm0001191/',
 'Scarlett Johansson': 'https://www.imdb.com/name/nm0424060/',
 'Sofia Vergara': 'https://www.imdb.com/name/nm0005527/',
 'Chris Evans.': 'https://www.imdb.com/name/nm0262635/'}
"""

os.environ['IMDB_DEBUG'] = '1'
del os.environ['IMDB_DEBUG']

for name, url in actors_url_dict.items():
    for name_t, url_t in actors_url_dict.items():
#         print(name, url, name_t, url_t)
        dis = get_movie_distance(url, url_t, 5, 5)
        print("{0}, {1}, {2}".format(name, name_t, dis))
#     break



Dwayne Johnson, Dwayne Johnson, 1
Dwayne Johnson, Chris Hemsworth, 2
Dwayne Johnson, Robert Downey Jr., 2
Dwayne Johnson, Akshay Kumar, inf
Dwayne Johnson, Jackie Chan, 2
Dwayne Johnson, Bradley Cooper, 2
Dwayne Johnson, Adam Sandler, 2
Dwayne Johnson, Scarlett Johansson, 2
Dwayne Johnson, Sofia Vergara, inf
Dwayne Johnson, Chris Evans, 2
Chris Hemsworth, Dwayne Johnson, 2
Chris Hemsworth, Chris Hemsworth, 1
Chris Hemsworth, Robert Downey Jr., 1


KeyboardInterrupt: 